In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain sentence-transformers faiss-cpu transformers torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 883.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import pickle
from transformers import pipeline
from pathlib import Path
import time

In [4]:
# Set up paths
VECTOR_STORE_DIR = Path('/content/drive/MyDrive/CrediTrust_Project/vector_store')
FAISS_INDEX_PATH = VECTOR_STORE_DIR / 'faiss_index.bin'
METADATA_PATH = VECTOR_STORE_DIR / 'metadata.pkl'
CHUNKS_PATH = VECTOR_STORE_DIR / 'chunks.pkl'

In [5]:
# Step 1: Load and verify vector store
if not all(p.exists() for p in [FAISS_INDEX_PATH, METADATA_PATH, CHUNKS_PATH]):
    raise FileNotFoundError("One or more vector store files missing")
index = faiss.read_index(str(FAISS_INDEX_PATH))
with open(METADATA_PATH, 'rb') as f:
    metadata = pickle.load(f)
with open(CHUNKS_PATH, 'rb') as f:
    chunks = pickle.load(f)

# Verify consistency
if index.ntotal != len(metadata) or index.ntotal != len(chunks):
    raise ValueError(f"File mismatch: FAISS={index.ntotal}, Metadata={len(metadata)}, Chunks={len(chunks)}")
print(f"Loaded {index.ntotal} chunks")

Loaded 27857 chunks


In [8]:
# Load embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')

In [9]:
# Load language model for generation
generator = pipeline('text-generation', model='distilgpt2', device=0)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [10]:
# Step 2: RAG pipeline
def rag_pipeline(query, k=5):
    start_time = time.time()

    # Encode query
    query_embedding = model.encode([query], show_progress_bar=False)

    # Retrieve top-k chunks, ensuring valid indices
    k = min(k, index.ntotal)  # Don't exceed index size
    distances, indices = index.search(np.array(query_embedding, dtype=np.float32), k)
    retrieved_chunks = [chunks[idx] for idx in indices[0] if idx < len(chunks)]
    retrieved_metadata = [metadata[idx] for idx in indices[0] if idx < len(metadata)]

    # Create context
    context = "\n".join([f"[{meta['product']}]: {chunk}" for chunk, meta in zip(retrieved_chunks, retrieved_metadata)])

    # Generate answer
    prompt = f"Question: {query}\nContext: {context}\nAnswer concisely:"
    response = generator(prompt, max_new_tokens=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    answer = response.split("Answer concisely:")[-1].strip()

    response_time = time.time() - start_time
    return answer, retrieved_chunks, retrieved_metadata, response_time

In [11]:
# Step 3: Evaluate with test questions
test_questions = [
    "Why are people unhappy with Buy Now, Pay Later (BNPL)?",
    "What are common issues with Credit card services?",
    "Are there complaints about Savings account fees?",
    "What problems do customers face with Money transfers?",
    "How do Personal loan complaints differ from BNPL?",
    "What are the main reasons for Credit card disputes?",
    "Do BNPL complaints involve high interest rates?",
    "Are there delays in Money transfer processing?"
]

evaluation_results = []
for query in test_questions:
    try:
        answer, chunks, metadata, response_time = rag_pipeline(query)
        answer_quality = 3  # Replace with manual rating (1-5)
        source_relevance = 3  # Replace with manual rating (1-5)
        evaluation_results.append({
            'Question': query,
            'Answer': answer,
            'Retrieved Products': [m['product'] for m in metadata],
            'Answer Quality (1-5)': answer_quality,
            'Source Relevance (1-5)': source_relevance,
            'Response Time (s)': round(response_time, 2)
        })
    except Exception as e:
        print(f"Error processing query '{query}': {e}")
        evaluation_results.append({
            'Question': query,
            'Answer': 'Error',
            'Retrieved Products': [],
            'Answer Quality (1-5)': 0,
            'Source Relevance (1-5)': 0,
            'Response Time (s)': 0
        })

# Save evaluation table
eval_df = pd.DataFrame(evaluation_results)
eval_df.to_csv('/content/evaluation_results.csv', index=False)
print("\nEvaluation Table:")
print(eval_df)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Evaluation Table:
                                            Question  \
0  Why are people unhappy with Buy Now, Pay Later...   
1  What are common issues with Credit card services?   
2   Are there complaints about Savings account fees?   
3  What problems do customers face with Money tra...   
4  How do Personal loan complaints differ from BNPL?   
5  What are the main reasons for Credit card disp...   
6    Do BNPL complaints involve high interest rates?   
7     Are there delays in Money transfer processing?   

                                              Answer  \
0  [Credit card]: not a day long payment but i ca...   
1  What are common issues with Credit card services?   
2  The question is: Can you explain in this way w...   
3  I would like to see a better way to evaluate t...   
4  I have written this for years and I have never...   
5  Credit card disputes are an ongoing issue in t...   
6  BNPL complaints involve high interest rates.\n...   
7  yes.\nThis is a problem, 

In [12]:
# Step 4: Validate BNPL representation
print("\nBNPL Validation:")
bnpl_count = sum(1 for m in metadata if m['product'] == 'Buy Now, Pay Later (BNPL)')
print(f"BNPL chunks: {bnpl_count} ({bnpl_count/len(metadata)*100:.2f}% of total)")




BNPL Validation:


ZeroDivisionError: division by zero

In [13]:
# Example usage
query = "Why are people unhappy with Buy Now, Pay Later (BNPL)?"
answer, chunks, metadata, response_time = rag_pipeline(query)
print(f"\nExample Query: {query}")
print(f"Answer: {answer}")
print(f"Retrieved Chunks: {chunks}")
print(f"Metadata: {metadata}")
print(f"Response Time: {response_time:.2f} seconds")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Example Query: Why are people unhappy with Buy Now, Pay Later (BNPL)?
Answer: because Buy Now is a free and open source software that is available for purchase in China.
Download this free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free software:
Download the free
Retrieved Chunks: []
Metadata: []
Response Time: 6.96 seconds
